# Python and Neo4J Playground

# Notes & Resources  
## py2neo  
Developed by the community, useful for data science  
https://nicolewhite.github.io/neo4j-jupyter/hello-world.html  
https://cr8ive.cf/2018/05/31/python-10-neo4j-graph-database-py2neo/  

## Neo4j driver
Officially supported by Neo4j, good if building apps to run in production  
https://neo4j.com/videos/get-started-w-pythonflask-and-neo4j-guest-roni-das-live-stream-by-michael-hunger/  
Configure drivers for security, etc.  
https://neo4j.com/developer/python/  

# Sample Code

## py2neo

In [6]:
from py2neo import Graph
from neographviz import Graph, plot

In [4]:
import getpass
password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")


Please enter the Neo4j database password to continue 
 ·······


In [7]:
graph = Graph(password=password)

In [8]:
plot(graph)

In [4]:
from IPython.display import IFrame
import json
import uuid

def vis_network(nodes, edges, physics=False):
    html = """
    <html>
    <head>
      <script type="text/javascript" src="../lib/vis/dist/vis.js"></script>
      <link href="../lib/vis/dist/vis.css" rel="stylesheet" type="text/css">
    </head>
    <body>
    <div id="{id}"></div>
    <script type="text/javascript">
      var nodes = {nodes};
      var edges = {edges};
      var container = document.getElementById("{id}");
      var data = {{
        nodes: nodes,
        edges: edges
      }};
      var options = {{
          nodes: {{
              shape: 'dot',
              size: 25,
              font: {{
                  size: 14
              }}
          }},
          edges: {{
              font: {{
                  size: 14,
                  align: 'middle'
              }},
              color: 'gray',
              arrows: {{
                  to: {{enabled: true, scaleFactor: 0.5}}
              }},
              smooth: {{enabled: false}}
          }},
          physics: {{
              enabled: {physics}
          }}
      }};
      var network = new vis.Network(container, data, options);
    </script>
    </body>
    </html>
    """

    unique_id = str(uuid.uuid4())
    html = html.format(id=unique_id, nodes=json.dumps(nodes), edges=json.dumps(edges), physics=json.dumps(physics))

    filename = "figure/graph-{}.html".format(unique_id)

    file = open(filename, "w")
    file.write(html)
    file.close()

    return IFrame(filename, width="100%", height="400")

def draw(graph, options, physics=False, limit=100):
    # The options argument should be a dictionary of node labels and property keys; it determines which property
    # is displayed for the node label. For example, in the movie graph, options = {"Movie": "title", "Person": "name"}.
    # Omitting a node label from the options dict will leave the node unlabeled in the visualization.
    # Setting physics = True makes the nodes bounce around when you touch them!
    query = """
    MATCH (n)
    WITH n, rand() AS random
    ORDER BY random
    LIMIT $limit
    OPTIONAL MATCH (n)-[r]->(m)
    RETURN n AS source_node,
           id(n) AS source_id,
           r,
           m AS target_node,
           id(m) AS target_id
    """

    data = graph.run(query, limit=limit)

    nodes = []
    edges = []

    def get_vis_info(node, id):
        node_label = list(node.labels())[0]
        prop_key = options.get(node_label)
        vis_label = node.properties.get(prop_key, "")

        return {"id": id, "label": vis_label, "group": node_label, "title": repr(node.properties)}

    for row in data:
        source_node = row[0]
        source_id = row[1]
        rel = row[2]
        target_node = row[3]
        target_id = row[4]

        source_info = get_vis_info(source_node, source_id)

        if source_info not in nodes:
            nodes.append(source_info)

        if rel is not None:
            target_info = get_vis_info(target_node, target_id)

            if target_info not in nodes:
                nodes.append(target_info)

            edges.append({"from": source_info["id"], "to": target_info["id"], "label": rel.type()})

    return vis_network(nodes, edges, physics=physics)

## Neo4j drivers

In [ ]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://76.251.77.235:7687", auth=('neo4j',password))
session=driver.session()

In [1]:
query="""
MATCH (a:Admissions)-[]-(d:Day)
RETURN a, d
LIMIT 5
"""
results=session.run(query)

NameError: name 'session' is not defined

In [5]:
from py2neo import Graph

graph = Graph(auth=('neo4j','M@rc3!M'))

In [6]:
from py2neo import Node

In [7]:
options = {"Patients": "SUBJECT_ID"}
draw(graph, options)

TypeError: 'LabelSetView' object is not callable